# Global Physical Constants and Configuration

## Detailed Analysis

This script serves as the centralized configuration file for the entire project. It defines a dictionary named `CONSTANTS` that contains physical constants, unit conversion factors, Equation of State (EoS) parameter ranges, numerical solver settings, and visualization limits. By referencing this file, consistency is maintained across the simulation workers, the machine learning pipeline, and the plotting scripts.

## Physics and Math

The project relies on specific unit systems and conversion factors to solve the Tolman-Oppenheimer-Volkoff (TOV) equations.

### Unit conversions
The TOV solver typically operates in geometric units or mixed units where convenient. The conversion between energy density (MeV/fm$^3$) and geometric length scales involves $\hbar c$:

$$
\hbar c \approx 197.33 \text{ MeV fm}
$$

This constant is used to convert pressures and energy densities into dimensionless or geometric forms suitable for integration.

### General Relativity Factors
To solve the TOV equations, mass and pressure terms are scaled by gravitational constants:

*   **G_CONV**: Relates energy density to curvature ($\sim G/c^4$).
*   **A_CONV**: Relates mass to length units ($\sim G/c^2$).

### Quark Stability Window
For the Quark Star model (Generalized CFL), the parameters are constrained by the stability of nuclear matter. The vacuum pressure $B$ (Bag Constant) and the superconducting gap $\Delta$ are bounded to ensure that:

1.  Quark matter is stable relative to neutrons at high densities.
2.  Ordinary nuclei (e.g., Iron) do not spontaneously decay into quark matter at zero pressure.

These constraints dictate the ranges `Q_B_MIN`, `Q_DELTA_RANGE`, and `Q_MS_RANGE`.

## Code Walkthrough

The `CONSTANTS` dictionary is organized into logical sections.

### 1. Fundamental Physics & Units
This section defines the core physical constants used throughout the simulation.

```python
'HC': 197.33,
'M_N': 939.0,
'G_CONV': 1.124e-5,
'A_CONV': 1.4766,
```

*   `HC`: The value of $\hbar c$ in MeV·fm.
*   `M_N`: The mass of a neutron in MeV.
*   `G_CONV` and `A_CONV`: Pre-calculated factors used in the `tov_rhs.py` script to couple matter terms to the metric.

### 2. Crust Physics (SLy Model)
These parameters define the transition thresholds for the multi-layer hadronic crust model based on the Douchin & Haensel (SLy) EoS.

```python
'P_C1': 9.34375e-5,
'P_C2': 4.1725e-8,
# ...
'P_TRANS_DEFAULT': 0.184,
```

*   `P_C1`, `P_C2`, `P_C3`: Pressure thresholds (MeV/fm$^3$) separating different crust layers (e.g., envelope, outer crust, inner crust).
*   `P_TRANS_DEFAULT`: The pressure at which the crust transitions to the core fluid.

### 3. Numerical Solver Settings
This block controls the behavior of the ODE integrator (RK45) in `solve_sequence.py`.

*   `TOV_R_MIN`: The starting radius (to avoid the singularity at $r=0$).
*   `TOV_P_MIN_SAFE`: A floor value for pressure to prevent numerical errors (e.g., logarithms of zero).

### 4. Quark and Hadronic Model Parameters
These ranges define the prior distribution for the Monte Carlo generation of EoS curves.

*   `Q_B_MIN`: The minimum allowed Vacuum Pressure $B$ for stability.
*   `Q_DELTA_RANGE`: The sampled range for the gap parameter $\Delta$.
*   `H_M_MIN_TARGET`: The minimum target mass for hadronic stars, used to filter out soft EoS models that cannot support massive stars.
*   `CAUSALITY_EPS_LIMIT`: A density threshold used to detect and clamp causality violations ($v_s > c$).

### 5. Plotting Standards
These constants enforce consistent axis limits across all visualization scripts.

```python
'PLOT_R_LIM': (5.0, 20.0),
'PLOT_M_LIM': (0.0, 4.0),
# ...
'BUCHDAHL_FACTOR': 2.25,
```

*   `PLOT_..._LIM`: Tuples defining $(min, max)$ for matplotlib axes.
*   `BUCHDAHL_FACTOR`: Represents the coefficient $9/4$ in the Buchdahl limit $R > \frac{9}{4} \frac{GM}{c^2}$.

### 6. Data Schema
The `COLUMN_SCHEMA` list defines the columns for the output CSV file (`thesis_dataset.csv`). This ensures that the generation workers and the training scripts expect the exact same data structure.

*   **Macro features**: `Mass`, `Radius`, `Lambda`.
*   **Micro features**: `Eps_Central`, `CS2_Central`.
*   **Topological features**: `Slope14` (slope of M-R curve at 1.4 solar masses).
*   **Model parameters**: `Bag_B`, `Gap_Delta` (specific to Quark models).